In [1]:
#imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF, metrics
from sklearn.metrics import make_scorer
from sklearn_crfsuite.metrics import flat_classification_report

In [2]:
st = "C:\\Users\\XLMR3003\\Documents\\movie.csv"
df = pd.read_csv(st,encoding = "ISO-8859-1" ,error_bad_lines=False)
#df[:10]

In [3]:
def add_sentence_index(object):
    sent_index=[]
    j=0
    for i in range(len(df)):
        if (df['Tag'][i]!='blank_line'):
            sent_index.insert(i,j)
        elif(df['Tag'][i]=='blank_line'):
            sent_index.insert(i,j)
            #print(j)
            j=j+1
        
    df['sentence_index']=sent_index
    #df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
    #df.reset_index(drop=True, inplace=True)
    print(df[:10])

In [4]:
add_sentence_index(df)

          Tag    Word  sentence_index
0           O    what               0
1           O  movies               0
2           O    star               0
3     B-ACTOR   bruce               0
4     I-ACTOR  willis               0
5  blank_line     NaN               0
6           O    show               1
7           O      me               1
8           O   films               1
9           O    with               1


In [5]:
df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
df.reset_index(drop=True, inplace=True)
import nltk
t =nltk.pos_tag(df['Word'].values.tolist())
t1=[]
for i in range(len(t)):
    t1.insert(i,t[i][1])

df['pos']=t1
df.head()
    

,Tag,Word,sentence_index,pos
0,O,what,0,WP
1,O,movies,0,NNS
2,O,star,0,VBP
3,B-ACTOR,bruce,0,NN
4,I-ACTOR,willis,0,NN


In [6]:
def complete_sentence_getter(df):
    y=lambda s : [str(w) for w in s['Word'].values.tolist()]
    
    grouped = df.groupby("sentence_index").apply(y)
    
    sentences= [" ".join(s) for s in grouped]
    return sentences
    
    

In [7]:
t=complete_sentence_getter(df)
l=[]
for i in t:
    l.append([i])

In [8]:
def complete_sentence_getter1(df):
    y=lambda s : [str(w) for w in s['Word'].values.tolist()]
    
    grouped = df.groupby("sentence_index").apply(y)
    return grouped

In [9]:
r = complete_sentence_getter1(df)


In [10]:
r[0]

['what', 'movies', 'star', 'bruce', 'willis']

In [11]:
wordfreq = {}
for word in df['Word']:
    if word not in wordfreq:
        wordfreq[word] = 0 
    wordfreq[word] += 1

In [12]:
class sentence(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        self.empty = False
        agg = lambda s : [(str(w), 
                           p,
                           t) for w, p, t in zip(s['Word'].values.tolist(),
                                                       s['pos'].values.tolist(),
                                                       s['Tag'].values.tolist())]
        self.grouped = self.df.groupby("sentence_index").apply(agg)
        self.sentences = [s for s in self.grouped]
        
    def get_text(self):
        try:
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent +=1
            return s
        except:
            return None

In [13]:
getter = sentence(df)
sentences = getter.sentences

In [14]:
def word2features(sent, i):
    word = str(sent[i][0])
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        #'w2v': [model[word]],
        'word_len' : len(word),
        'postag[:2]': postag[:2],
        'word_freq': int(wordfreq[word]/100),
        
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            #'w2v': [model[word]],
            'word_len' : len(word),
           '-1:postag[:2]': postag1[:2],
            'word_freq': int(wordfreq[word]/100),
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            #'w2v': [model[word]],
            'word_len' : len(word),
            '+1:postag[:2]': postag1[:2],
            'word_freq': int(wordfreq[word]/100),
        })
    else:
        features['EOS'] = True

    return features


In [15]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for 
            token,
            postag,
            
            label 
            in sent]

def sent2tokens(sent):
    return [token for token, 
            postag, 
            label in sent]

In [16]:
X = [sent2features(s) for s in sentences]
z = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]


In [17]:
X[0]

[{'bias': 1.0,
  'word.lower()': 'what',
  'word[-3:]': 'hat',
  'word[-2:]': 'at',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit()': False,
  'postag': 'WP',
  'word_len': 4,
  'postag[:2]': 'WP',
  'word_freq': 90,
  'BOS': True,
  '+1:word.lower()': 'movies',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:postag': 'NNS',
  '+1:postag[:2]': 'NN'},
 {'bias': 1.0,
  'word.lower()': 'movies',
  'word[-3:]': 'ies',
  'word[-2:]': 'es',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit()': False,
  'postag': 'NNS',
  'word_len': 6,
  'postag[:2]': 'NN',
  'word_freq': 22,
  '-1:word.lower()': 'what',
  '-1:word.istitle()': False,
  '-1:word.isupper()': False,
  '-1:postag': 'WP',
  '-1:postag[:2]': 'WP',
  '+1:word.lower()': 'star',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:postag': 'VBP',
  '+1:postag[:2]': 'VB'},
 {'bias': 1.0,
  'word.lower()': 'star',
  'word[-3:]': 'tar',
  'word[-2:]': 'ar',
  'wo

In [18]:

t
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [19]:
from timeit import default_timer as timer
crf = CRF(algorithm = 'lbfgs',
         c1= 0.519154,
         c2=0.017584,     
         max_iterations = 150,
         all_possible_transitions = True, verbose = True)

start = timer()
crf.fit(X_train, y_train)
end = timer()
print("Time elapsed")
print (end - start)

loading training data to CRFsuite: 100%|███████████████████████████████████████| 17587/17587 [00:07<00:00, 2267.87it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 108191
Seconds required: 2.796

L-BFGS optimization
c1: 0.519154
c2: 0.017584
num_memories: 6
max_iterations: 150
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=25.32 loss=847741.08 active=101031 feature_norm=0.12
Iter 2   time=6.21  loss=782699.68 active=99920 feature_norm=0.12
Iter 3   time=22.84 loss=744605.27 active=99077 feature_norm=0.12
Iter 4   time=9.46  loss=737386.53 active=99927 feature_norm=0.12
Iter 5   time=4.76  loss=734792.49 active=99891 feature_norm=0.13
Iter 6   time=5.08  loss=729054.54 active=99695 feature_norm=0.14
Iter 7   time=4.47  loss=714729.11 active=99409 feature_norm=0.18
Iter 8   time=4.40  loss=699482.73 active=99768 feature_norm=0.28
Iter 9   time=4.46  loss=688294.01 active=99796 feature_norm=0.26
Ite

KeyboardInterrupt: 

In [ ]:
y_pred = crf.predict(X_train)
report = flat_classification_report(y_train, y_pred)
print(report)

In [ ]:
y_pred = crf.predict(X_test)
report = flat_classification_report(y_test, y_pred)
print(report)

In [ ]:
y_pred = crf.predict(X_test)
report = flat_classification_report(y_test, y_pred)
print(report)

In [ ]:
# define fixed parameters and parameters to search
import scipy
from sklearn_crfsuite import metrics
from sklearn.model_selection import RandomizedSearchCV
crf3 = CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True,
    verbose=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted')

# search
rs = RandomizedSearchCV(crf3, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 84.7min


In [ ]:
print('Best parameters:', rs.best_params_)
print('Best CV score:', rs.best_score_)
print('Model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

In [ ]:
crf3 = rs.best_estimator_
y_pred = crf3.predict(X)
print(metrics.flat_classification_report(
    y, y_pred, labels=sorted_labels, digits=3))

In [ ]:
c1: 0.519154
c2: 0.017584

In [ ]:
import scipy
from sklearn.model_selection import RandomizedSearchCV
crf = CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted')

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=10,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

In [ ]:
y_pred = crf.predict(X_test)